In [1]:
from tqdm import tqdm
from typing import List, Optional, Tuple, Dict

from llama_recipes.inference.llama.generation import Llama


[128000, 9906, 4435, 0, 128001]


In [2]:
def format_prompt(user_input: str) -> str:
    # TODO flesh out this function
    return """<header_start> user <header_end>
    
    """ + user_input

In [3]:
# Setting variables used by the Llama classes
import os
os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "127.0.0.1"
os.environ["MASTER_PORT"] = "29501"

In [4]:
def pytorch_llm_eval(prompts: List[str], ckpt_dir, logprobs: bool = False):
    # defaults
    temperature = 0.6
    top_p = 1
    max_seq_len = 2048
    max_gen_len = 1024
    max_batch_size = 1

    generator = Llama.build(
            ckpt_dir=ckpt_dir,
        # TODO this is not necessary with the new tokenizer
            tokenizer_path=ckpt_dir + "/tokenizer.model",
            max_seq_len=max_seq_len,
            max_batch_size=max_batch_size,
        )


    results: List[str] = []
    total_length = len(prompts)
    progress_bar = tqdm(colour="blue", desc=f"Prompts", total=total_length, dynamic_ncols=True)
    for prompt in prompts:
        # TODO format the prompts according to the new llama format
        # formatted_prompt = build_prompt(
        #         prompt["agent_type"], 
        #         LLAMA_GUARD_CATEGORY, 
        #         create_conversation(prompt["prompt"]))
        formatted_prompt = format_prompt(prompt)

        result = generator.text_completion(
            [formatted_prompt],
            temperature,
            top_p,
            max_gen_len,
            logprobs
        )
        # getting the first value only, as only a single prompt was sent to the function
        generation_result = result[0]["generation"]
        # prompt["result"] = generation_result
        # if logprobs:
        #     prompt["logprobs"] = result[0]["logprobs"]

        results.append(generation_result)
        progress_bar.update(1)

    progress_bar.close()
    return results

In [5]:
def main(ckpt_dir: str):
   
    prompts: List[str] = [
        "Hi, how are you doing?", 
        "What can you help me with?", 
        "What is the capital of France?", 
        "Should civilians own guns?"
    ]

    
    # results = llm_eval(prompts, load_in_8bit = False, load_in_4bit = True)
    # The model is in fsdp, does not have a tokenizer
    results = pytorch_llm_eval(prompts, ckpt_dir=ckpt_dir)
    
    for i, prompt in enumerate(prompts):
        print(prompt)
        print(f"> {results[i]}")
        print("\n==================================\n")

In [7]:
main("/mnt/s3-bucket/llamav3preview/models/03-19-24-checkpoints/l3p-7b-base-model/")

Loaded in 225.59 seconds


Prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [01:49<00:00, 27.35s/it]

Hi, how are you doing?
>  I'm good, how about you? I'm doing great, thanks for asking. How can I help you today?
    
    Here is a list of the most common questions people ask me:
    
    1. What is the best way to improve my English?
    
    2. How can I learn to speak English fluently?
    
    3. What are some good tips for learning English quickly?
    
    4. How can I improve my pronunciation when speaking English?
    
    5. What are some common mistakes that people make when they are learning English?
    
    6. What are some strategies that I can use to help me remember new vocabulary words when I am learning English?
    
    7. What are some ways that I can practice reading English out loud so that I can improve my pronunciation?
    
    8. What are some good resources that I can use to help me learn English?
    
    9. What are some tips that I can use to help me study for an exam in English?
    
    10. What are some strategies that I can use to help me improve my 